In [1]:
import os
os.getcwd()

'C:\\Users\\Dell'

In [2]:
#Although this code is giving error but it successfully imports english stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

NameError: name 'nltk' is not defined

In [4]:
BASE_DIR = '/Users/Dell/20_newsgroups'
LABELS = ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']

def get_dataset():
    with open('data.txt', 'w', encoding='utf8') as outfile:
        
#storing files labelwise...like all 'alt.theism' comes at 0 index of 'train_file_labelled' dictionary and so on
        train_file_labelled = {}

#storing each file of training data individually in 'train_file_unlabelled' dictionary without takingcare of their labels
        train_file_unlabelled = {}

#storing each file of testing data individually in 'test_file' dictionary without takingcare of their labels
        test_file = {}

#storing the labels in numeric forms in below lists for each training and testing documents
        y_train_labelled = []
        y_train_unlabelled = []
        y_test = []
        
        k_test_idx = 0
        k_train_unlabel_idx = 0
        p = -1
        unlabel_p = -1
        
#traversing the entire document labelwise
        for label in LABELS:
            dir = '%s/%s' % (BASE_DIR, label)
            i = 0
            p = p + 1

#storing all training documents labelwise inside 'train_file_labelled' dictionary
            train_file_labelled[label] = {}
    
            unlabel_p = unlabel_p + 1
            k_train_idx = 0
            
            for filename in os.listdir(dir):
                fullfilename = '%s/%s' % (dir, filename)
                train_file_labelled[label][k_train_idx] = []
                test_file[k_test_idx] = []
                with open(fullfilename, 'rb') as file:
                    if(i < 750):
                        text = file.read().decode(errors='replace').replace('\n','')
                        
#storing current text document labelwise
                        train_file_labelled[label][k_train_idx] = text
                        k_train_idx = k_train_idx+1
                        y_train_labelled.append(p)
                        i = i+1
                
#storing current text document but not labelwise        
                        train_file_unlabelled[k_train_unlabel_idx] = text
                        y_train_unlabelled.append(unlabel_p)
                        k_train_unlabel_idx = k_train_unlabel_idx+1
                        
                        outfile.write('%s\t%s\t%s\n' % (label, filename, text))
                    else:
                        text = file.read().decode(errors='replace').replace('\n','')
                        test_file[k_test_idx] = text
                        y_test.append(p)
                        k_test_idx = k_test_idx+1
                        i = i+1
                        outfile.write('%s\t%s\t%s\n' % (label, filename, text))
                        
    return train_file_labelled, y_train_labelled, test_file, y_test, train_file_unlabelled, y_train_unlabelled

In [5]:
train_data_labelled, y_train_labelled, test_data, y_test, train_data_unlabelled, y_train_unlabelled = get_dataset()

In [6]:
def char_to_word_labelled_data(data):
    class_in_words = {}
#         traversing each class
    for curr_key in data.keys():
        class_in_words[curr_key] = []
#         traversing each folder of a class
        for curr_key_key in data[curr_key].keys():
            data_str = ""
#             traversing single page
            for curr_str in data[curr_key][curr_key_key]:
                if(curr_str.isalpha()):
                    data_str+=curr_str
                elif(data_str != ""):
                    class_in_words[curr_key].append(data_str)
                    data_str = ""
#             class_in_words[curr_key].append(data_str)
    return class_in_words

In [7]:
# documents are dtored in the form of string of characters which needs to be converted to string of words.
#This is being done using the below code.
train_data_labelled_in_word = char_to_word_labelled_data(train_data_labelled)

In [8]:
def char_to_word_unlabelled_data(data):
    class_in_words = {}
#         traversing each class
    for curr_key in data.keys():
        class_in_words[curr_key] = []
#         traversing each folder of a class
#         for curr_key_key in data[curr_key].keys():
        data_str = ""
#             traversing single page
        for curr_str in data[curr_key]:
            if(curr_str.isalpha()):
                data_str+=curr_str
            elif(data_str != ""):
                class_in_words[curr_key].append(data_str)
                data_str = ""
#             class_in_words[curr_key].append(data_str)
    return class_in_words

In [9]:
# documents are dtored in the form of string of characters which needs to be converted to string of words.
#This is being done using the below code.
test_data_in_word = char_to_word_unlabelled_data(test_data)
train_data_unlabelled_in_word = char_to_word_unlabelled_data(train_data_unlabelled)

In [10]:
def dictionary_for_unlabelled_docs(data_fit):
    result = {}
    for curr_class in data_fit.keys():
        for word in data_fit[curr_class]:
            if(word not in stopwords.words('english')):
                if(word in result.keys()):
                    result[word] = result[word] + 1
                else:
                    result[word] = 1
    return result

In [11]:
def dictionary_for_labelled_docs(data_fit):
    dictionary = {}
    for curr_class in data_fit.keys():
        dictionary[curr_class] = {}
        for word in data_fit[curr_class]:
            if(word not in stopwords.words('english')):
                if(word in dictionary[curr_class].keys()):
                    dictionary[curr_class][word] = dictionary[curr_class][word] + 1
                else:
                    dictionary[curr_class][word] = 1
    return dictionary

In [12]:
# creating dictionary for labelled training documents
labelled_train_dictionary = dictionary_for_labelled_docs(train_data_labelled_in_word)

In [14]:
# creating dictionary for unlabelled training documents
unlabelled_train_dictionary = dictionary_for_unlabelled_docs(train_data_labelled_in_word)

In [15]:
# sorting the unlabelled_train_dictionary in descending order to get first few topmost important words 
import operator
sorted_dictionary = dict( sorted(unlabelled_train_dictionary.items(), key=operator.itemgetter(1),reverse=True))

In [18]:
def top_n_words_dictionary(modified_dictionary):
    new_dictionary = {}
    i = 0
    for key in modified_dictionary.keys():
        new_dictionary[key] = modified_dictionary[key]
        i = i+1
        if(i==4000):
            break
    return new_dictionary

In [19]:
# getting the dictionary which contains the key-value pair for top n words only
top_n_words_dictionary = top_n_words_dictionary(sorted_dictionary)

In [ ]:
# getting the top n words
final_n_words = top_n_words_dictionary.keys()

In [20]:
def find_single_doc_word_count(doc):
    word_count = {}
    for label_word in final_n_words:
        for word in doc:
            if(word == label_word):# and (word in final_words)):
                if(word in word_count.keys()):
                    word_count[word] = word_count[word]+1
                else:
                    word_count[word] = 1
    return word_count

In [21]:
def doc_word_count(unlabelled_data):
    count_word = {}
    for i in range(len(unlabelled_data)):
        count_word[i] = {}
        count_word[i] = find_single_doc_word_count(unlabelled_data[i])
    return count_word

In [22]:
# finding the word counts for each documents in train_data_labelled_in_word 
train_each_labelled_doc_dict = doc_word_count(train_data_labelled_in_word)

In [37]:
def convert_to_2d_array(data_each_dict):
    array = []
    i = 0
    for key in data_each_dict.keys():
        temp = []
        for word in final_n_words:
#             array[i] = []
#             for single_word in train_each_doc_dict[key].keys():
            if(word in data_each_dict[key].keys()):
                temp.append(data_each_dict[key][word])
            else:
                temp.append(0)
        array.append(temp)
        i = i+1
    return array

In [38]:
# converting the train data into 2d array
train_array = convert_to_2d_array(train_each_labelled_doc_dict)

In [25]:
# finding the word counts for each documents in test_data_in_word
test_each_doc_dict = doc_word_count(test_data_in_word)

In [43]:
# converting the test data into 2d array
test_array = convert_to_2d_array(test_each_doc_dict)

In [26]:
def create_X_data(dictnry,final_dictnry):
    curr_row = 0
    rows, cols = (20, 4000)
    X = [[0 for i in range(cols)] for j in range(rows)]
    for dictnry_key in dictnry.keys():
        curr_col = 0
        for key in final_dictnry.keys():
            if key in dictnry[dictnry_key].keys():
                X[curr_row][curr_col] = dictnry[dictnry_key][key]
            curr_col = curr_col + 1
        curr_row = curr_row + 1
    return X

In [23]:
# the below code will create a 2d list which contains count of top n words for each documents
xtrain = create_X_data(labelled_train_dictionary,top_n_words_dictionary)

In [ ]:
import numpy as np
xtrain = np.array(xtrain)

In [25]:
# calculating the probability of each word for each document in xtrain and storing in a 2d numpy array xtrain_probability
xtrain_probability = xtrain / xtrain.sum(axis = 0)

array([[0.05952193, 0.06184959, 0.04876875, ..., 0.04201681, 0.05042017,
        0.04201681],
       [0.04886323, 0.04111205, 0.05206222, ..., 0.04201681, 0.05042017,
        0.02521008],
       [0.0476954 , 0.0822241 , 0.04699534, ..., 0.05042017, 0.01680672,
        0.01680672],
       ...,
       [0.05338623, 0.06667473, 0.0594852 , ..., 0.18487395, 0.07563025,
        0.1512605 ],
       [0.05748288, 0.0553001 , 0.05953587, ..., 0.04201681, 0.11764706,
        0.06722689],
       [0.05896583, 0.05666142, 0.06614816, ..., 0.03361345, 0.02521008,
        0.11764706]])

In [27]:
def predict_single_doc_class(doc,xtrain_probability,final_dictionary):
    i = 0
    max_prob = -1
    prob = [0 for i in range(20)]
    for this_class in LABELS:
        k = 0
#         prob.append(0)
        for this_word in final_dictionary.keys():
            if this_word in doc:
                prob[i] += xtrain_probability[i][k]
            k = k+1
        if(prob[i]>max_prob):
            max_prob = prob[i]
            resultant_class = i
        i = i+1
    return resultant_class

In [28]:
def predict_all_doc(data):
    ypredict = []
    for this_doc in data.keys():
        ypredict.append(predict_single_doc_class(np.array(test_data_final[this_doc]),xtrain_probability,top_n_words_dictionary))
    return ypredict

In [29]:
# predicting the classess for each test document
ypredicted = predict_all_doc(test_data_in_word)

In [1]:
ypredicted = np.array(ypredicted)
ytest = np.array(y_test)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("Classification Report Using Own code from Scratch")
print(classification_report(ytest, ypredicted))
print("Confusion Matrix")
print(confusion_matrix(ytest, ypredicted))
print()
print("Accuracy Score")
print(accuracy_score(ytest, ypredicted) * 100, "%", sep="")

NameError: name 'np' is not defined

<!--Result for #top_words = 4000 using manual implemenatation without sklearn is as follows:-->
Classification Report
              precision    recall  f1-score   support

           0       0.75      0.77      0.76       250
           1       0.94      0.78      0.85       250
           2       0.61      0.99      0.76       250
           3       0.88      0.89      0.89       250
           4       0.99      0.85      0.92       250
           5       0.90      0.84      0.87       250
           6       0.97      0.74      0.84       250
           7       0.98      0.82      0.89       250
           8       1.00      0.86      0.92       250
           9       0.99      0.88      0.93       250
          10       0.93      0.98      0.95       250
          11       0.77      0.98      0.86       250
          12       0.99      0.56      0.71       250
          13       1.00      0.83      0.91       250
          14       0.94      0.94      0.94       250
          15       0.94      1.00      0.97       247
          16       0.82      0.85      0.84       250
          17       0.63      0.99      0.77       250
          18       0.60      0.75      0.67       250
          19       0.69      0.53      0.60       250

    accuracy                           0.84      4997
   macro avg       0.87      0.84      0.84      4997
weighted avg       0.87      0.84      0.84      4997

Confusion Matrix
[[192   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0   8
    5  44]
 [  2 194  22   1   0  10   0   0   0   0   0   8   0   0   2   0   0   5
    6   0]
 [  0   0 248   0   0   2   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]
 [  2   2  17 222   0   3   0   0   0   0   0   2   0   0   0   0   0   2
    0   0]
 [  0   0   8   9 213   0   2   0   0   0   0   2   0   0   0   0   0  10
    6   0]
 [  0   3  33   1   0 211   0   0   0   0   0   1   0   0   0   0   0   0
    1   0]
 [  0   0  28  11   1   1 185   1   0   2   1   4   1   0   1   0   1   5
    7   1]
 [  0   1   9   0   0   2   2 204   1   1   0   2   0   0   1   0   4   4
   19   0]
 [  0   0  10   0   0   0   1   2 214   0   0   2   0   0   0   0   2  13
    5   1]
 [  0   0   5   0   0   0   0   0   0 221  11   0   0   0   0   1   0   6
    6   0]
 [  0   0   3   0   0   0   0   0   0   0 244   1   0   0   1   0   0   0
    1   0]
 [  0   0   2   0   0   0   0   0   0   0   0 245   0   0   1   0   1   0
    1   0]
 [  1   2  13   7   0   2   0   2   0   0   1  43 139   1   6   0   3  24
    6   0]
 [ 10   3   4   0   0   2   0   0   0   0   2   2   0 208   1   0   0   7
   11   0]
 [  1   1   0   0   1   0   0   0   0   0   0   0   0   0 235   0   0   5
    7   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 247   0   0
    0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   4   0   0   0   0 213  13
   18   2]
 [  0   0   1   0   0   0   0   0   0   0   0   1   0   0   0   1   0 247
    0   0]
 [  0   0   0   0   0   0   0   0   0   0   2   0   0   0   0   0  27  23
  188  10]
 [ 47   0   1   0   0   0   0   0   0   0   0   3   0   0   1  13   9  17
   27 132]]

Accuracy Score
84.09045427256355%

In [47]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(train_array,y_train_unlabelled)
ypred_sk = clf.predict(test_array)

In [52]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("Classification Report Using Sklearn Multinomial NaiveBayes")
print(classification_report(y_test,ypred_sk))
print("Confusion Matrix")
print(confusion_matrix(y_test,ypred_sk))
print()
print("Accuracy Score")
print(accuracy_score(y_test,ypred_sk))

              precision    recall  f1-score   support

           0       0.77      0.68      0.72       250
           1       0.70      0.75      0.73       250
           2       0.91      0.34      0.50       250
           3       0.61      0.84      0.71       250
           4       0.77      0.92      0.84       250
           5       0.84      0.77      0.80       250
           6       0.73      0.91      0.81       250
           7       0.86      0.88      0.87       250
           8       0.90      0.95      0.92       250
           9       0.92      0.92      0.92       250
          10       0.96      0.92      0.94       250
          11       0.95      0.88      0.91       250
          12       0.74      0.83      0.78       250
          13       0.96      0.77      0.86       250
          14       0.91      0.90      0.91       250
          15       0.94      1.00      0.97       247
          16       0.83      0.89      0.86       250
          17       0.90    

<!--Result for #top_words = 4000 using sklearn is as follows:-->
Classification Report
              precision    recall  f1-score   support

           0       0.77      0.68      0.72       250
           1       0.70      0.75      0.73       250
           2       0.91      0.34      0.50       250
           3       0.61      0.84      0.71       250
           4       0.77      0.92      0.84       250
           5       0.84      0.77      0.80       250
           6       0.73      0.91      0.81       250
           7       0.86      0.88      0.87       250
           8       0.90      0.95      0.92       250
           9       0.92      0.92      0.92       250
          10       0.96      0.92      0.94       250
          11       0.95      0.88      0.91       250
          12       0.74      0.83      0.78       250
          13       0.96      0.77      0.86       250
          14       0.91      0.90      0.91       250
          15       0.94      1.00      0.97       247
          16       0.83      0.89      0.86       250
          17       0.90      0.89      0.90       250
          18       0.82      0.71      0.76       250
          19       0.62      0.67      0.64       250

    accuracy                           0.82      4997
   macro avg       0.83      0.82      0.82      4997
weighted avg       0.83      0.82      0.82      4997
[[169   0   0   0   0   0   3   4   1   0   0   0   5   0   1   2   0   1
    1  63]
 [  1 188   1   7  14   9   8   1   2   1   0   7   5   0   6   0   0   0
    0   0]
 [  0  24  86  89  18  21   4   0   1   0   0   0   7   0   0   0   0   0
    0   0]
 [  1   4   1 211  19   4   5   0   0   0   0   0   5   0   0   0   0   0
    0   0]
 [  0   1   0   8 229   0  11   0   0   0   0   0   0   0   1   0   0   0
    0   0]
 [  0  26   3  12   6 193   3   0   3   1   0   0   0   1   2   0   0   0
    0   0]
 [  0   0   0   8   0   0 228   1   1   1   1   0   8   1   1   0   0   0
    0   0]
 [  0   0   2   2   1   1   8 221   4   1   2   0   6   0   1   0   0   0
    1   0]
 [  0   0   0   1   0   0   9   2 237   0   0   0   1   0   0   0   0   0
    0   0]
 [  0   0   1   0   1   0   6   2   4 229   4   0   0   1   0   0   2   0
    0   0]
 [  0   0   0   0   1   0   3   0   2  10 231   0   0   0   3   0   0   0
    0   0]
 [  0   5   0   0   0   2   4   1   3   1   0 220  11   0   1   0   2   0
    0   0]
 [  0   7   0   7   2   1   3   4   0   0   0   1 207   1   3   0   0   9
    5   0]
 [  0   5   0   0   4   0  12   7   2   4   2   1  18 193   1   0   0   0
    1   0]
 [  0   5   0   0   1   0   1   2   0   0   0   0   5   4 226   0   0   0
    6   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 247   0   0
    0   0]
 [  0   0   0   1   1   0   1   7   0   0   0   2   0   0   0   0 222   0
    5  11]
 [  5   1   0   0   1   0   1   2   3   0   0   1   0   0   1   4   2 222
    4   3]
 [  0   1   0   0   0   0   1   1   0   0   0   0   1   0   0   0  31  12
  177  26]
 [ 43   1   0   0   0   0   1   1   0   0   0   0   0   0   1   9   9   2
   16 167]]
Accuracy: 0.821092655593356 = 82.10 %

Therefore manual implementation gives better result than using sklearn 